<a href="https://colab.research.google.com/github/t0r8ern1t/comp_mod/blob/main/mm_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import csv

In [2]:
def read_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        file.readline()
        for line in file:
            station, lon, lat, depth, ve, vn = map(float, line.split())
            data.append((station, lon, lat, depth, ve, vn))
    return data

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371000
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def rectangle_integration(f_values, dx, method='left'):
    if method == 'left':
        return sum(f_values[:-1]) * dx
    elif method == 'right':
        return sum(f_values[1:]) * dx

def trapezoidal_integration(f_values, dx):
    return (f_values[0] + 2 * sum(f_values[1:-1]) + f_values[-1]) * dx / 2

def simpson_integration(f_values, dx):
    if len(f_values) < 3 or len(f_values) % 2 == 0:
        return trapezoidal_integration(f_values, dx)
    n = len(f_values)
    integral = f_values[0] + f_values[-1]
    for i in range(1, n - 1, 2):
        integral += 4 * f_values[i]
    for i in range(2, n - 2, 2):
        integral += 2 * f_values[i]
    return integral * dx / 3

def integrate_depth(data, station):
    depths = [entry[3] for entry in data if entry[0] == station]
    ve_values = [entry[4] for entry in data if entry[0] == station]
    vn_values = [entry[5] for entry in data if entry[0] == station]

    if len(depths) <= 1:
        return 0, 0, 0, 0, 0, 0, 0, 0

    dx = abs(depths[1] - depths[0])

    ve_integral_trap = trapezoidal_integration(ve_values, dx)
    vn_integral_trap = trapezoidal_integration(vn_values, dx)

    ve_integral_rect_left = rectangle_integration(ve_values, dx, method='left')
    vn_integral_rect_left = rectangle_integration(vn_values, dx, method='left')

    ve_integral_rect_right = rectangle_integration(ve_values, dx, method='right')
    vn_integral_rect_right = rectangle_integration(vn_values, dx, method='right')

    ve_integral_simp = simpson_integration(ve_values, dx)
    vn_integral_simp = simpson_integration(vn_values, dx)

    return (ve_integral_trap, vn_integral_trap,
            ve_integral_rect_left, vn_integral_rect_left,
            ve_integral_rect_right, vn_integral_rect_right,
            ve_integral_simp, vn_integral_simp)

def integrate_space(data):
    stations = sorted(set(entry[0] for entry in data))
    total_ve_trap, total_vn_trap = 0, 0
    total_ve_rect_left, total_vn_rect_left = 0, 0
    total_ve_rect_right, total_vn_rect_right = 0, 0
    total_ve_simp, total_vn_simp = 0, 0

    for i in range(len(stations) - 1):
        station1, station2 = stations[i], stations[i+1]

        lat1, lon1 = next((entry[2], entry[1]) for entry in data if entry[0] == station1)
        lat2, lon2 = next((entry[2], entry[1]) for entry in data if entry[0] == station2)
        dx = calculate_distance(lat1, lon1, lat2, lon2)

        integrals1 = integrate_depth(data, station1)
        integrals2 = integrate_depth(data, station2)

        total_ve_trap += trapezoidal_integration([integrals1[0], integrals2[0]], dx)
        total_vn_trap += trapezoidal_integration([integrals1[1], integrals2[1]], dx)

        total_ve_rect_left += trapezoidal_integration([integrals1[2], integrals2[2]], dx)
        total_vn_rect_left += trapezoidal_integration([integrals1[3], integrals2[3]], dx)

        total_ve_rect_right += trapezoidal_integration([integrals1[4], integrals2[4]], dx)
        total_vn_rect_right += trapezoidal_integration([integrals1[5], integrals2[5]], dx)

        total_ve_simp += trapezoidal_integration([integrals1[6], integrals2[6]], dx)
        total_vn_simp += trapezoidal_integration([integrals1[7], integrals2[7]], dx)

    return {
        "ve_trap": total_ve_trap, "vn_trap": total_vn_trap,
        "ve_rect_left": total_ve_rect_left, "vn_rect_left": total_vn_rect_left,
        "ve_rect_right": total_ve_rect_right, "vn_rect_right": total_vn_rect_right,
        "ve_simp": total_ve_simp, "vn_simp": total_vn_simp
    }

def write_results_to_csv(results, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ["Method", "Ve", "Vn"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerow({"Method": "Trapezoidal", "Ve": results["ve_trap"], "Vn": results["vn_trap"]})
        writer.writerow({"Method": "Rectangle Left", "Ve": results["ve_rect_left"], "Vn": results["vn_rect_left"]})
        writer.writerow({"Method": "Rectangle Right", "Ve": results["ve_rect_right"], "Vn": results["vn_rect_right"]})
        writer.writerow({"Method": "Simpson", "Ve": results["ve_simp"], "Vn": results["vn_simp"]})

In [4]:
file_path = r"Data_3.txt"
output_file = r"output.csv"

data = read_data(file_path)

results = integrate_space(data)

write_results_to_csv(results, output_file)

print(f"Результаты сохранены в файл: {output_file}")

Результаты сохранены в файл: output.csv
